In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import EqualityDataset
from equality_experiment import EqualityExperiment


params = dict(
    embed_dims=[10],
    hidden_dims=[100],
    alphas=[0.001],
    learning_rates=[0.01],
    n_trials=2,
    train_sizes=list(range(104, 2005, 100))
)

experiment = EqualityExperiment(    
    dataset_class=EqualityDataset,
    **params)

In [3]:
experiment.run_once()

Using the first item in grid: embed_dim=10 hidden_dim=100 alpha=0.001 lr=0.01 ...
creating dataset
creating dataset
shape of X_train: (2004, 20)
shape of X_test: (500, 20)
Accuracy: 1.000
number of layers: 2
shape of weights in first layer: (20, 100)
shape of weights in second layer: (100, 1)
shape of bias in first layer: (100,)
shape of bias in second layer: (1,)


In [2]:
from torch_equality import *

params = dict(
    embed_dims=[10],
    hidden_dims=[100],
    alphas=[0.001],
    learning_rates=[0.01],
    n_trials=2,
    train_sizes=list(range(1004, 2005, 1000))
)

experiment = TorchEqualityExperiment(dataset_class=TorchEqualityDataset, **params)

In [3]:
experiment.run()

Grid size: 1 * 2; 2 experiments
Running trials for embed_dim=10 hidden_dim=100 alpha=0.001 lr=0.01 ... mean: 0.83; max: 0.994; took 6.0 secs


,trial,train_size,embed_dim,hidden_dim,alpha,learning_rate,accuracy,batch_pos,batch_neg
0,1,0,10,100,0.001,0.01,0.462,0,0
1,1,1004,10,100,0.001,0.01,0.984,505,499
2,1,2004,10,100,0.001,0.01,0.994,1002,1002
3,2,0,10,100,0.001,0.01,0.554,0,0
4,2,1004,10,100,0.001,0.01,0.992,489,515
5,2,2004,10,100,0.001,0.01,0.992,1002,1002


In [3]:
embed_dim = 10
max_epochs = 100
hidden_dim = 100
train_size = 2000
test_size = 500
alpha = 0.001
lr = 0.01

train_dataset = TorchEqualityDataset(embed_dim=embed_dim, n_pos=train_size//2, n_neg=train_size//2)
test_dataset = TorchEqualityDataset(embed_dim=embed_dim, n_pos=test_size//2, n_neg=test_size//2)

train_dataset.test_disjoint(test_dataset)
model = TorchEqualityModel(max_epochs=max_epochs,
                           input_size=embed_dim*2,
                           batch_size=1000,
                           hidden_layer_size=hidden_dim,
                           alpha=alpha,
                           lr=lr,
                           gpu=True)

model.fit(train_dataset)


In [5]:
print(test_dataset.X[0], test_dataset.y[0])
print(test_dataset.X[3], test_dataset.y[3])

[ 0.39201654 -0.36785673  0.44480057 -0.29828635  0.09587019  0.21976774
 -0.21790579  0.47005808  0.13934583 -0.39761212 -0.14932567  0.43585482
  0.16314534 -0.20044318  0.34226107 -0.28727885 -0.36907869  0.08117762
  0.23866461 -0.261373  ] 0
[ 0.05293649 -0.0405006  -0.00809856  0.17018614 -0.06961887 -0.10660474
  0.27717561  0.24940513 -0.19529317 -0.4902961   0.05293649 -0.0405006
 -0.00809856  0.17018614 -0.06961887 -0.10660474  0.27717561  0.24940513
 -0.19529317 -0.4902961 ] 1


In [21]:
from intervention_interface import *

intervention = TorchEqualityIntervention(model)

In [22]:
res_3 = intervention.run(torch.tensor(test_dataset.X[3]))
print(res_3)
res_0 = intervention.run(torch.tensor(test_dataset.X[0]))
print(res_0)

[1]
[0]


In [25]:
res_3 = intervention.run(torch.tensor(test_dataset.X[3]))
print(intervention.get_from_cache("hidden_vec"))
res_intervention = intervention.fix_and_run("hidden_vec", torch.tensor(test_dataset.X[0]))
print(res_3, res_intervention)

tensor([0.0000e+00, 0.0000e+00, 9.8200e-04, 0.0000e+00, 0.0000e+00, 7.3625e-02,
        2.1076e-04, 0.0000e+00, 7.7771e-04, 0.0000e+00, 8.6266e-04, 5.7836e-04,
        0.0000e+00, 1.1789e-03, 0.0000e+00, 0.0000e+00, 5.5889e-04, 8.8341e-03,
        0.0000e+00, 0.0000e+00, 4.0467e-04, 1.1502e-04, 0.0000e+00, 0.0000e+00,
        1.1389e-04, 2.2425e-01, 0.0000e+00, 2.1191e-01, 0.0000e+00, 3.5294e-01,
        3.4319e-02, 0.0000e+00, 1.1725e-01, 6.4332e-01, 1.8010e-01, 0.0000e+00,
        0.0000e+00, 4.9733e-05, 4.8089e-02, 0.0000e+00, 0.0000e+00, 2.0517e-01,
        0.0000e+00, 0.0000e+00, 1.3748e-01, 2.7138e-01, 1.9672e-04, 0.0000e+00,
        1.2978e-01, 0.0000e+00, 0.0000e+00, 7.5374e-05, 0.0000e+00, 2.9675e-01,
        2.5111e-04, 1.0445e-01, 0.0000e+00, 6.2687e-03, 1.2862e-01, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0108e-03, 0.0000e+00, 0.0000e+00, 1.4319e-01,
        2.1146e-01, 1.8739e-01, 0.0000e+00, 1.6255e-03, 0.0000e+00, 0.0000e+00,
        2.8291e-01, 0.0000e+00, 8.7655e-